In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips, TextClip
import numpy as np

runnersite = "https://runalyze.com/athlete/Schrottie"


In [ ]:
# Funktion zum Abrufen der Webseite und Extrahieren der relevanten Informationen
def extract_activity_data():
    response = requests.get(runnersite)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    activities = []
    today = datetime.today().strftime('%d.%m.%Y')
    
    # Finden aller Zeilen mit Laufaktivitäten
    activity_rows = soup.find_all('tr', class_='r')
    
    data = {
        'date': [],
        'a_type': [],
        'r_type': [],
        'distance': [],
        'duration': [],
        'pace': []
    }
    
    current_date = None
    
    for row in activity_rows:
        cols = row.find_all('td')
        
        if cols:
            date_cell = cols[1].text.strip()
            if date_cell == '': # komplett leer bedeutet, dass es nicht die erste Aktivität des Tages ist
                offset = 0
            else:
                offset = 1
                new_date = date_cell.split()[0] + '.' + str(datetime.today().year)
                # Das Datum noch "merken", damit es in Zeilen ohne Datum genutzt weden kann
                if new_date and new_date != current_date:
                    current_date = new_date
                    
            if row.find('i', class_='icons8-Running'):
                rt_str = cols[3 + offset].text.strip() if len(cols) > 3 + offset else ''
                distance_str = cols[4 + offset].text.strip().split()[0].replace(',', '.') if len(cols) > 4 + offset else ''
                duration_str = cols[5 + offset].text.strip() if len(cols) > 5 + offset else ''
                pace_str = cols[6 + offset].text.strip() if len(cols) > 6 + offset else ''
                
                # Überprüfen, ob die aktuellen Werte nicht leer sind
                if distance_str and duration_str and pace_str:
                    activities.append({'date': current_date, 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
                else:
                    # Verwenden des vorherigen Datums, wenn die aktuellen Werte leer sind
                    if data['date']:
                        activities.append({'date': data['date'][-1], 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
    
    df = pd.DataFrame(activities)
    df = df[df['a_type'] == 'run'] # Filtern nach Aktivitätstyp "run"
    # Dataframe aufräumen, damit Datentypen passen und später keine Fehler bewirken
    df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
    df['pace'] = df['pace'].str.replace("/km", "")  # Entferne "/km"
    df['pace'] = df['pace'].apply(lambda x: datetime.strptime(x, '%M:%S'))
    df['duration'] = pd.to_timedelta(df['duration'])
    df['duration_minutes'] = df['duration'].dt.total_seconds() / 60
    
    return df

In [ ]:
# Funktion zur Erstellung des Diagramms
def create_chart(activities):
    # Anzahl der Zeilen im Dataframe
    n_rows = activities.shape[0]

    # Erstelle ein Figure-Objekt mit quadratischen Abmessungen
    fig, ax = plt.subplots(figsize=(10, 10))

    # Setze die Beschriftung der Y-Achse auf "von - bis"
    #ax.set_ylabel(f"{activities['date'].min()} bis {activities['date'].max()}")
    
    # Erstelle einen horizontalen Balkenplot für die Distanz
    for i, (dist, dur, pace) in enumerate(zip(activities["distance"], activities["duration"], activities["pace"])):
        if i % 2 == 0:
            color = mcolors.to_rgba_array(plt.cm.bone(0.3 + i * 0.3 / n_rows))
        else:
            color = mcolors.to_rgba_array(plt.cm.pink(0.3 + i * 0.3 / n_rows))
        ax.barh(i, dist, color=color, edgecolor='black')

        # Formatieren der Dauer, damit kein "0 days" vorangestellt ist
        dur_formatted = str(dur).split()[-1]

        # Beschrifte die Balken mit den Distanzwerten am rechten Rand des Balkens
        ax.text(dist, i, f"{dur_formatted} / {pace}  ", ha='right', va='center', rotation=0, color='white')

    # Setze die Beschriftungen der X-Achse für die Distanz
    #ax.set_xlabel("Laufdistanz (km)")
    
    # Entferne die Beschriftungen der Y-Achse
    ax.set_yticks([])

    # Hintergrundfarbe setzen
    ax.set_facecolor('#fffff0')

    # Zeige das Diagramm an
    plt.show()

In [ ]:
# Funktion zur Erstellung des Videos
def create_video(activities):
    # Videoauflösung
    width, height = 1920, 1080
    duration_per_row = 3  # Dauer für jede Zeile in Sekunden
    fade_duration = 1  # Überblendungsdauer in Sekunden
    final_duration = 7  # Dauer der Abschlussanzeige in Sekunden

    # Erstelle das Hintergrundbild für das Diagramm
    fig, ax = plt.subplots(figsize=(width / 100, height / 100))
    ax.axis('off')  # Achsen entfernen
    ax.set_xlim([0, 10])  # Beispielgrenzen für x-Achse
    ax.set_ylim([0, 10])  # Beispielgrenzen für y-Achse
    ax.text(5, 5, "Hier kommt das Diagramm", ha='center', va='center', fontsize=24)
    fig.savefig('chart.png', bbox_inches='tight', dpi=100)  # Diagramm als Bild speichern
    plt.close()

    # Erstelle den Textclip für die Abschlussanzeige
    sum_text = f"Gesamt: Distanz {activities['distance'].sum()} km, Dauer {activities['duration'].sum()}, Pace {activities['pace'].mean()}"
    final_text_clip = (TextClip(sum_text, fontsize=70, color='white', bg_color='black')
                       .set_duration(final_duration)
                       .set_position(('center', 'center')))

    # Erstelle den Clip für das Diagramm
    chart_clip = (ImageClip('chart.png')
                  .set_duration(duration_per_row * len(activities))
                  .resize(height=height // 2)  # Setze Höhe auf die Hälfte der Videohöhe
                  .set_position(('center', 'bottom')))

    # Erstelle die Clips für die Datenzeilen
    data_clips = []
    for i, (_, row) in enumerate(activities.iterrows()):
        text = f"Datum: {row['date']} - Distanz: {row['distance']} km - Dauer: {row['duration']} - Pace: {row['pace']}"
        text_clip = (TextClip(text, fontsize=40, color='white', bg_color='black')
                     .set_duration(duration_per_row)
                     .set_position(('center', 'top')))
        data_clips.append(text_clip)

    # Füge Fade-Übergänge zwischen den Clips hinzu
    clips_with_fades = [data_clips[0]]
    for i in range(1, len(data_clips)):
        clip_with_fade = concatenate_videoclips([data_clips[i-1].fadeout(fade_duration), data_clips[i].fadein(fade_duration)])
        clips_with_fades.append(clip_with_fade)

    # Füge die Abschlussanzeige am Ende hinzu
    final_clip = concatenate_videoclips(clips_with_fades + [final_text_clip])

    # Erstelle das Gesamtvideo
    video = concatenate_videoclips([chart_clip, final_clip])
    video = video.resize(width=width)  # Setze die Breite auf die gewünschte Videoauflösung

    # Speichere das Video
    video.write_videofile("activity_video.mp4", fps=24, codec='libx264')

In [ ]:
# Hauptfunktion
def main():
    activities = extract_activity_data()
    if not activities.empty:  # Check if activities is not empty
        create_chart(activities)
        create_video(activities)
        #print("Video wurde erfolgreich erstellt.")
    else:
        print("Keine Aktivitäten gefunden.")

if __name__ == "__main__":
    main()